## Capstone Project - Columbia DSI - DiDi - Spring 2021
#### Authors:  Danyang Han,  Anita Pinto,  Elizabeth Yum
This file is used to generate the matching probability based on a given demand   
Reference: https://github.com/skyetim/didi-vehicle-repositioning-strategy(Fall 2020 Capstone Project) in particular EDA_activetime_dist

In [1]:
import pandas as pd 
import datetime as dt 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from sqlalchemy import create_engine

In [2]:
df = pd.read_csv('../data/trip_cleaned.csv') 
print(df.head(5))

    medallion  hack_license vendor_id  rate_code      pickup_datetime  \
0  2013000001    2013000001       VTS          1  2013-06-24 18:55:00   
1  2013000001    2013000001       VTS          1  2013-06-24 19:12:00   
2  2013000001    2013000001       VTS          1  2013-06-24 19:45:00   
3  2013000001    2013000001       VTS          1  2013-06-24 20:17:00   
4  2013000001    2013000001       VTS          1  2013-06-24 20:30:00   

      dropoff_datetime  passenger_count  trip_distance  pickup_longitude  \
0  2013-06-24 19:01:00                1           0.97        -73.992531   
1  2013-06-24 19:30:00                1           3.04        -73.982498   
2  2013-06-24 20:10:00                2           4.17        -73.981850   
3  2013-06-24 20:28:00                1           2.35        -73.988129   
4  2013-06-24 20:35:00                1           1.24        -73.961098   

   pickup_latitude  ...  trip_time     delta  last_dropoff  first_pickup  \
0        40.724052  ...     

In [3]:
df.columns

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'pickup_taxizone_id',
       'dropoff_taxizone_id', 'pickup_datetime_next', 'dropoff_datetime_next',
       'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount',
       'tolls_amount', 'total_amount', 'trip_time', 'delta', 'last_dropoff',
       'first_pickup', 'dropoff_time', 'pickup_time', 'dropoff_day',
       'pickup_day', 'pickup_weekday', 'dropoff_weekday'],
      dtype='object')

In [6]:
df['pickup_time_act'] = pd.to_datetime(df['pickup_datetime']).dt.time
df['dropoff_time_act'] = pd.to_datetime(df['dropoff_datetime']).dt.time
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

In [7]:
df = df.sort_values(['hack_license', 'pickup_datetime', 'dropoff_datetime'])

In [8]:
df.head(10)

,medallion,hack_license,vendor_id,rate_code,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,...,last_dropoff,first_pickup,dropoff_time,pickup_time,dropoff_day,pickup_day,pickup_weekday,dropoff_weekday,pickup_time_act,dropoff_time_act
0,2013000001,2013000001,VTS,1,2013-06-24 18:55:00,2013-06-24 19:01:00,1,0.97,-73.992531,40.724052,...,0,0.0,19:00:00,19:00:00,2013-06-24,2013-06-24,1,1,18:55:00,19:01:00
1,2013000001,2013000001,VTS,1,2013-06-24 19:12:00,2013-06-24 19:30:00,1,3.04,-73.982498,40.745617,...,0,0.0,19:30:00,19:15:00,2013-06-24,2013-06-24,1,1,19:12:00,19:30:00
2,2013000001,2013000001,VTS,1,2013-06-24 19:45:00,2013-06-24 20:10:00,2,4.17,-73.981850,40.768753,...,0,0.0,20:15:00,19:45:00,2013-06-24,2013-06-24,1,1,19:45:00,20:10:00
3,2013000001,2013000001,VTS,1,2013-06-24 20:17:00,2013-06-24 20:28:00,1,2.35,-73.988129,40.738098,...,0,0.0,20:30:00,20:15:00,2013-06-24,2013-06-24,1,1,20:17:00,20:28:00
4,2013000001,2013000001,VTS,1,2013-06-24 20:30:00,2013-06-24 20:35:00,1,1.24,-73.961098,40.760826,...,0,0.0,20:30:00,20:30:00,2013-06-24,2013-06-24,1,1,20:30:00,20:35:00
5,2013000001,2013000001,VTS,1,2013-06-24 20:37:00,2013-06-24 20:49:00,2,2.60,-73.955383,40.776684,...,0,0.0,20:45:00,20:30:00,2013-06-24,2013-06-24,1,1,20:37:00,20:49:00
6,2013000001,2013000001,VTS,1,2013-06-24 20:52:00,2013-06-24 21:03:00,1,2.36,-73.979774,40.781242,...,0,0.0,21:00:00,20:45:00,2013-06-24,2013-06-24,1,1,20:52:00,21:03:00
7,2013000001,2013000001,VTS,1,2013-06-24 22:07:00,2013-06-24 22:13:00,2,0.82,-73.929214,40.753971,...,1,0.0,22:15:00,22:00:00,2013-06-24,2013-06-24,1,1,22:07:00,22:13:00
8,2013000001,2013000001,VTS,1,2013-06-26 17:24:00,2013-06-26 17:35:00,1,0.86,-73.971581,40.757946,...,0,1.0,17:30:00,17:30:00,2013-06-26,2013-06-26,1,1,17:24:00,17:35:00
9,2013000001,2013000001,VTS,1,2013-06-26 17:36:00,2013-06-26 17:49:00,2,1.23,-73.972939,40.748775,...,0,0.0,17:45:00,17:30:00,2013-06-26,2013-06-26,1,1,17:36:00,17:49:00


In [15]:
df['next_pickup_zone'] = np.where(df['last_dropoff'] == 0, df['pickup_taxizone_id'].shift(-1), 0)   

In [21]:
pd.set_option('display.max_columns', None)
df.head(10)

,medallion,hack_license,vendor_id,rate_code,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime_next,dropoff_datetime_next,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,trip_time,delta,last_dropoff,first_pickup,dropoff_time,pickup_time,dropoff_day,pickup_day,pickup_weekday,dropoff_weekday,pickup_time_act,dropoff_time_act,next_pickup_zone
0,2013000001,2013000001,VTS,1,2013-06-24 18:55:00,2013-06-24 19:01:00,1,0.97,-73.992531,40.724052,-73.983841,40.731033,148.0,79.0,2013-06-24 19:12:00,2013-06-24 19:30:00,CSH,6.0,1.0,0.5,0.00,0.0,7.50,6.0,0.183333,0,0.0,19:00:00,19:00:00,2013-06-24,2013-06-24,1,1,18:55:00,19:01:00,164.0
1,2013000001,2013000001,VTS,1,2013-06-24 19:12:00,2013-06-24 19:30:00,1,3.04,-73.982498,40.745617,-73.982628,40.777344,164.0,143.0,2013-06-24 19:45:00,2013-06-24 20:10:00,CRD,14.5,1.0,0.5,0.00,0.0,16.00,18.0,0.250000,0,0.0,19:30:00,19:15:00,2013-06-24,2013-06-24,1,1,19:12:00,19:30:00,142.0
2,2013000001,2013000001,VTS,1,2013-06-24 19:45:00,2013-06-24 20:10:00,2,4.17,-73.981850,40.768753,-73.987068,40.726727,142.0,79.0,2013-06-24 20:17:00,2013-06-24 20:28:00,CRD,17.5,1.0,0.5,3.70,0.0,22.70,25.0,0.116667,0,0.0,20:15:00,19:45:00,2013-06-24,2013-06-24,1,1,19:45:00,20:10:00,234.0
3,2013000001,2013000001,VTS,1,2013-06-24 20:17:00,2013-06-24 20:28:00,1,2.35,-73.988129,40.738098,-73.965042,40.761581,234.0,141.0,2013-06-24 20:30:00,2013-06-24 20:35:00,CRD,10.5,0.5,0.5,2.20,0.0,13.70,11.0,0.033333,0,0.0,20:30:00,20:15:00,2013-06-24,2013-06-24,1,1,20:17:00,20:28:00,141.0
4,2013000001,2013000001,VTS,1,2013-06-24 20:30:00,2013-06-24 20:35:00,1,1.24,-73.961098,40.760826,-73.956833,40.774902,141.0,263.0,2013-06-24 20:37:00,2013-06-24 20:49:00,CSH,6.0,0.5,0.5,0.00,0.0,7.00,5.0,0.033333,0,0.0,20:30:00,20:30:00,2013-06-24,2013-06-24,1,1,20:30:00,20:35:00,263.0
5,2013000001,2013000001,VTS,1,2013-06-24 20:37:00,2013-06-24 20:49:00,2,2.60,-73.955383,40.776684,-73.978905,40.786716,263.0,239.0,2013-06-24 20:52:00,2013-06-24 21:03:00,CRD,10.5,0.5,0.5,2.20,0.0,13.70,12.0,0.050000,0,0.0,20:45:00,20:30:00,2013-06-24,2013-06-24,1,1,20:37:00,20:49:00,239.0
6,2013000001,2013000001,VTS,1,2013-06-24 20:52:00,2013-06-24 21:03:00,1,2.36,-73.979774,40.781242,-73.948364,40.781536,239.0,263.0,2013-06-24 22:07:00,2013-06-24 22:13:00,CSH,10.0,0.5,0.5,0.00,0.0,11.00,11.0,1.066667,0,0.0,21:00:00,20:45:00,2013-06-24,2013-06-24,1,1,20:52:00,21:03:00,146.0
7,2013000001,2013000001,VTS,1,2013-06-24 22:07:00,2013-06-24 22:13:00,2,0.82,-73.929214,40.753971,-73.940140,40.749222,146.0,145.0,2013-06-26 17:24:00,2013-06-26 17:35:00,CSH,5.5,0.5,0.5,0.00,0.0,6.50,6.0,43.183333,1,0.0,22:15:00,22:00:00,2013-06-24,2013-06-24,1,1,22:07:00,22:13:00,0.0
8,2013000001,2013000001,VTS,1,2013-06-26 17:24:00,2013-06-26 17:35:00,1,0.86,-73.971581,40.757946,-73.972771,40.748951,162.0,233.0,2013-06-26 17:36:00,2013-06-26 17:49:00,CRD,8.0,1.0,0.5,1.80,0.0,11.30,11.0,0.016667,0,1.0,17:30:00,17:30:00,2013-06-26,2013-06-26,1,1,17:24:00,17:35:00,233.0
9,2013000001,2013000001,VTS,1,2013-06-26 17:36:00,2013-06-26 17:49:00,2,1.23,-73.972939,40.748775,-73.965302,40.757889,233.0,229.0,2013-06-26 17:56:00,2013-06-26 18:05:00,CRD,10.0,1.0,0.5,2.75,0.0,14.25,13.0,0.116667,0,0.0,17:45:00,17:30:00,2013-06-26,2013-06-26,1,1,17:36:00,17:49:00,233.0


In [32]:
zones = range(1,10,1)
taxi_after_dropoff = pd.DataFrame(columns=zones, index=zones)


for s1 in zones:
    for s2 in zones:
        events = df[(df['dropoff_taxizone_id']==s1) & (df['next_pickup_zone']==s2)]
        taxi_after_dropoff.loc[s1, s2] = len(events)

taxi_after_dropoff = taxi_after_dropoff.astype(float).div(taxi_after_dropoff.sum(axis=1), axis=0)
taxi_after_dropoff = taxi_after_dropoff.fillna(0)

In [33]:
taxi_after_dropoff.head(10)

,1,2,3,4,5,6,7,8,9
1,0.558140,0.0,0.000000,0.348837,0.0,0.0,0.093023,0.000000,0.000000
2,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
3,0.000000,0.0,0.846154,0.076923,0.0,0.0,0.076923,0.000000,0.000000
4,0.000000,0.0,0.000000,0.999301,0.0,0.0,0.000699,0.000000,0.000000
5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
6,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000
7,0.000076,0.0,0.000000,0.007077,0.0,0.0,0.992619,0.000228,0.000000
8,0.000000,0.0,0.000000,0.020833,0.0,0.0,0.541667,0.437500,0.000000
9,0.000000,0.0,0.000000,0.227273,0.0,0.0,0.409091,0.000000,0.363636


In [ ]:
zones = range(1,264,1)
taxi_after_dropoff = pd.DataFrame(columns=zones, index=zones)


for s1 in zones:
    for s2 in zones:
        events = df[(df['dropoff_taxizone_id']==s1) & (df['next_pickup_zone']==s2)]
        taxi_after_dropoff.loc[s1, s2] = len(events)

taxi_after_dropoff = taxi_after_dropoff.astype(float).div(taxi_after_dropoff.sum(axis=1), axis=0)
taxi_after_dropoff = taxi_after_dropoff.fillna(0)

In [ ]:
taxi_after_dropoff.head(10)